In [16]:
# ----------------------- Mạng Neural Nhân tạo để phân loại -------------------- #
# nhập thư viện bắt buộc
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [17]:
# Tải dữ liệu
bank_data = pd.read_csv("F:/University/DHQuyNhon/2.MachineLearning/ANN/artificial-neural-network-business_case_study-main/Artificial_Neural_Network_Case_Study_data.csv")
bank_data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [18]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [19]:
# Lấy tất cả các hàng và tất cả các cột trong dữ liệu ngoại trừ cột cuối cùng là X (ma trận tính năng)
# số hàng và id khách hàng không cần thiết cho việc lập mô hình, vì vậy chúng tôi loại bỏ và bắt đầu với điểm tín dụng
X = bank_data.iloc[:,3:-1].values
print("Independent variables are:", X)
# bỏ qua tất cả các hàng nhưng chỉ cột cuối cùng là Y (biến phụ thuộc)
y = bank_data.iloc[:, -1].values
print("Dependent variable is:", y)

Independent variables are: [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
Dependent variable is: [1 0 1 ... 1 1 0]


In [20]:
# Chuyển đổi biến giới tính, các nhãn được chọn ngẫu nhiên
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [21]:
# Chuyển đổi biến cột địa lý, các nhãn được chọn ngẫu nhiên, ct yêu cầu đối số [1] chỉ mục của vb đích
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(),[1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [22]:
# Tách dữ liệu thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# in kích thước của từng tập để xem số lượng hàng và cột 
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(8000, 12) (2000, 12)
(8000,) (2000,)


In [23]:
# Chia tỷ lệ dữ liệu / chuẩn hóa các tính năng sẽ chuyển sang NN
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
# ----------------------- Xây dựng mô hình ----------------------- #

In [25]:
# Khởi tạo ANN bằng cách gọi lớp Tuần tự từ keras của Tensorflow
ann = tf.keras.models.Sequential()

In [26]:
#----------------------------------------------------------------------------------
# Adding "fully connected" INPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
#----------------------------------------------------------------------------------
# Thêm input layer "fully connected" vào ANN tuần tự bằng cách gọi class Dense
#----------------------------------------------------------------------------------
# Số lượng đơn vị = 6 và Chức năng hoạt động = Chỉnh lưu
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [27]:
#----------------------------------------------------------------------------------
# Adding "fully connected" SECOND layer to the Sequential AMM by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [28]:
#----------------------------------------------------------------------------------
# Adding "fully connected" OUTPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 1 and Activation Function = Sigmoid
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [29]:
#----------------------- Training the model -----------------------#
# Compiling the ANN
# Type of Optimizer = Adam Optimizer, Loss Function =  crossentropy for binary dependent variable, and Optimization is done w.r.t. accuracy
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN model on training set  (fit method always the same)
# batch_size = 32, the default value, number of epochs  = 100
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5448 - accuracy: 0.7694
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4619 - accuracy: 0.8031
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4339 - accuracy: 0.8094
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4200 - accuracy: 0.8129
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4092 - accuracy: 0.8150
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4000 - accuracy: 0.8175
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3921 - accuracy: 0.8183
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3854 - accuracy: 0.8195
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3794 - accuracy: 0.8202
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3743 - accura

In [30]:
#----------------------- Evaluating the Model ---------------------#
# the goal is to use this ANN model to predict the probability of the customer leaving the bank
# Predicting the churn probability for single observations
#----------------------- Đánh giá Mô hình---------------------#
# mục tiêu là sử dụng mô hình ANN này để dự đoán xác suất khách hàng rời khỏi ngân hàng
# Dự đoán xác suất rời bỏ cho các quan sát đơn lẻ

#Geography: French
#Credit Score:600
#Gender: Male
#Age: 40 years old
#Tenure: 3 years
#Balance: $60000
#Number of Products: 2
#with Credit Card
#Active member
#Estimated Salary: $50000

In [31]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# this customer has 4% chance to leave the bank

1/1 [==============================] - 0s 142ms/step
[[0.0336156]]
1/1 [==============================] - 0s 44ms/step
[[False]]


In [32]:
#show the vector of predictions and real values
#probabilities
# hiển thị vectơ của các dự đoán và giá trị thực
# xác suất
y_pred_prob = ann.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [33]:
# xác suất để nhị phân
y_pred = (y_pred_prob > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [34]:
#Confusion Matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix", confusion_matrix)
print("Accuracy Score", accuracy_score(y_test, y_pred))

Confusion Matrix [[1503   92]
 [ 192  213]]
Accuracy Score 0.858
